# Json데이터를 SQL에 넣기 위해 진행하는 과정

### 필수 패키지 및 경고 무시 진행

In [1]:
#사용한 import
import warnings
import pandas as pd
import json
import random
import pymysql


# 특정 경고 무시
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=FutureWarning)

### JSON 데이터 불러오기

In [ ]:
#JSON 파일 불러오기
#중분류 가공 JSON 데이터 불러오기
with open('/Users/isanghyeon/Desktop/json/test.json', 'r', encoding='utf-8') as f:
    card_detail_data = json.load(f)
#체크카드 JSON 데이터 불러오기
with open('/Users/isanghyeon/Desktop/json/check_cards.json', 'r', encoding='utf-8') as f:
    check_cards = json.load(f)
#신용카드 JSON 데이터 불러오기
with open('/Users/isanghyeon/Desktop/json/credit_cards.json', 'r', encoding='utf-8') as f:
    credit_cards = json.load(f)
#카드 홈페이지 바로가기 링크 불러오기
with open('/Users/isanghyeon/Desktop/json/link.json', 'r', encoding='utf-8') as f:
    card_url = json.load(f)

### 카테고리 설정 및 이미지 링크 설정

In [2]:
#값저장하기

#대카테고리 데이터 생성
categories = ['모든가맹점','모빌리티','대중교통','통신','생활','쇼핑','외식/카페','뷰티/피트니스','금융/포인트','병원/약국','문화/취미','숙박/항공']

#소카테고리명
all_com = ["모든가맹점"]
car = ["주유소","자동차/하이패스"]
public = ["기차","택시","고속버스","대중교통","교통"]
call = ["LGU+","KT","SKT","통신"]
life = ["공과금/렌탈","교육/육아","애완동물","교육/육아"]
shopping = ['편의점','온라인쇼핑','마트/백화점','쇼핑']
food = ['음식점','배달','카페','디저트','외식/카페']
beauty = ['뷰티','피트니스','뷰티/피트니스']
point = ['혜택','금융','보험','간편결제','해외']
hospital = ['병원','약국','병원/약국']
ott = ['OTT','영화','취미','OTT/영화/문화']
air = ['항공','공항라운지/PP','프리미엄',"여행/숙박"]
#대-중 카테고리 연결
arr_key2 = ['모든가맹점','모빌리티','대중교통','통신','생활','쇼핑','외식/카페','뷰티/피트니스','금융/포인트','병원/약국','문화/취미','숙박/항공']
arr_value2 = [all_com,car,public,call,life,shopping,food,beauty,point,hospital,ott,air]


#혜택 아이콘 링크
# sts에서의 경로
benefit_images = {
    "모든가맹점": "/resources/images/benefitdetailImage/all.png",
    "주유소": "/resources/images/benefitdetailImage/GasStation.png",
    "자동차/하이패스": "/resources/images/benefitdetailImage/Car.png",
    "기차": "/resources/images/benefitdetailImage/Train.png",
    "택시": "/resources/images/benefitdetailImage/Taxi.png",
    "고속버스": "/resources/images/benefitdetailImage/Bus.png",
    "대중교통": "/resources/images/benefitdetailImage/public.png",
    "교통": "/resources/images/benefitdetailImage/public.png",
    "LGU+": "/resources/images/benefitdetailImage/LGU.png",
    "KT": "/resources/images/benefitdetailImage/KT.png",
    "SKT": "/resources/images/benefitdetailImage/SKT.png",
    "통신": "/resources/images/benefitdetailImage/call.png",
    "공과금/렌탈": "/resources/images/benefitdetailImage/Bill.png",
    "교육/육아": "/resources/images/benefitdetailImage/BookAndPencil.png",
    "생활": "/resources/images/benefitdetailImage/BookAndPencil.png",
    "애완동물": "/resources/images/benefitdetailImage/Pets.png",
    "편의점": "/resources/images/benefitdetailImage/Shop.png",
    "온라인쇼핑": "/resources/images/benefitdetailImage/OnlineShopping.png",
    "마트/백화점": "/resources/images/benefitdetailImage/mart.png",
    "쇼핑": "/resources/images/benefitdetailImage/shopping.png",
    "음식점": "/resources/images/benefitdetailImage/Restaurant.png",
    "배달": "/resources/images/benefitdetailImage/Delivery.png",
    "카페": "/resources/images/benefitdetailImage/Cafe.png",
    "디저트": "/resources/images/benefitdetailImage/Dessert.png",
    "외식/카페": "/resources/images/benefitdetailImage/cafeAndDes.png",
    "뷰티": "/resources/images/benefitdetailImage/hair.png",
    "피트니스": "/resources/images/benefitdetailImage/Strength.png",
    "뷰티/피트니스": "/resources/images/benefitdetailImage/hair.png",
    "혜택": "/resources/images/benefitdetailImage/GetCash.png",
    "금융": "/resources/images/benefitdetailImage/CreditCard.png",
    "보험": "/resources/images/benefitdetailImage/Outpatient.png",
    "간편결제": "/resources/images/benefitdetailImage/SmartphonePay.png",
    "해외": "/resources/images/benefitdetailImage/Earth.png",
    "병원": "/resources/images/benefitdetailImage/Hospital.png",
    "약국": "/resources/images/benefitdetailImage/Hospital.png",
    "병원/약국": "/resources/images/benefitdetailImage/Hospital.png",
    "OTT": "/resources/images/benefitdetailImage/Netflix.png",
    "영화": "/resources/images/benefitdetailImage/MovieProjector.png",
    "취미": "/resources/images/benefitdetailImage/Bowling.png",
    "OTT/영화/문화": "/resources/images/benefitdetailImage/MovieProjector.png",
    "항공": "/resources/images/benefitdetailImage/Airport.png",
    "공항라운지/PP": "/resources/images/benefitdetailImage/Cocktail.png",
    "프리미엄": "/resources/images/benefitdetailImage/Guarantee.png",
    "여행/숙박": "/resources/images/benefitdetailImage/Luggage.png"
}
# 이미지 링크 데이터 베이스 저장
df_link = pd.DataFrame(list(benefit_images.items()), columns=['benefitdetail_name', 'benefitdetail_image'])

### 데이터 가공 시작

#### 금액 부분 '원'이란 글자를 포함하면 해당 글자 제거 후 숫자만 남기기

In [ ]:
def clean_fee(value):
    if not value or value == "없음":
        return 0
    # '원' 제거 및 숫자만 추출
    return int(''.join(filter(str.isdigit, value)))

#### 체크카드/신용카드 JSON 데이터 가공

In [ ]:
#체크카드/신용카드 JSON 합치기 진행
card_type_data = check_cards + credit_cards

#2개의 JSON Dataframe 형태로 변환
df_type = pd.DataFrame(card_type_data)
df_url = pd.DataFrame(card_url)

#### JSON 데이터를 통해 세부적인 card table 값 설정

In [ ]:
records = []
for card in card_detail_data:
    records.append({
        "card_id": card.get('id'),
        "card_name": card.get('card_name'),
        "company": card.get('company'),
        "brand": card.get('type'),
        "card_image": card.get('main_image'),
        "fee_domestic": card.get('fees', {}).get('domestic'),
        "fee_foreign": card.get('fees', {}).get('foreign'),
        "prev_month_cost": int(card.get('prev_month_cost')[:2]) 
            if card.get('prev_month_cost') 
            and card.get('prev_month_cost') != '없음' 
            and card.get('prev_month_cost')[:2].isdigit() 
            else 0,
        "card_like": random.randint(0,20000),
        "card_views": random.randint(0,40000)
    })
df_card = pd.DataFrame(records)

#### 위에서 가공한 데이터프레임들 합치기 시작

In [ ]:
#카드데이터와 신용/체크 & 카드사바로가기 데이터 merge
# card_name 기준 중복 제거
df_type_unique = df_type.drop_duplicates(subset="card_name")
# company 기준 중복 제거
df_url_unique = df_url.drop_duplicates(subset="company")

df_card_final = pd.merge(df_card, df_type_unique, on="card_name", how="left")
#df_card_final = df_card_final.rename(columns={"card_type": "신용/체크"})
df_card_final['fee_foreign'] = df_card_final['fee_foreign'].apply(clean_fee)
df_card_final['fee_domestic'] = df_card_final['fee_domestic'].apply(clean_fee)

df_card_final = pd.merge(df_card_final, df_url_unique, on="company", how="left")
df_card_final = df_card_final[["card_id","card_name","company","card_type","brand","card_image","card_url"
                  ,"fee_domestic","fee_foreign","prev_month_cost","card_like","card_views"]]
df_card_final['card_type'] = df_card_final['card_type'].fillna("신용카드")

#### 카테고리 세부 설정

In [ ]:
# 1차 대카테고리 값 숫자 매핑
benefitcategory_map = {
    1:"모든가맹점",
    2:"모빌리티",
    3:"대중교통",
    4:"통신",
    5:"생활",
    6:"쇼핑",
    7:"외식/카페",
    8:"뷰티/피트니스",
    9:"금융/포인트",
    10:"병원/약국",
    11:"문화/취미",
    12:"숙박/항공"
}
#카테고리명 랜덤 번호 부여
category_id_map = {i+1:cat for i, cat in enumerate(categories)}
df_bigCategories = pd.DataFrame(list(benefitcategory_map.items()), columns=['benefitcategory_id', 'benefitCategory_name'])

#### 소카테고리 데이터 생성

In [ ]:
tag_dict = {}
for keywords, label in zip(arr_key2, arr_value2):
    tag_dict[keywords] = label

#대카테고리와 merge를 통해 값 연결
df_smallCategories3 = pd.DataFrame(list(tag_dict.items()), columns=['benefitCategory_name', 'benefitdetail_name'])
df_smallCategories2 = pd.merge(df_bigCategories, df_smallCategories3, on="benefitCategory_name", how="left")
#리스트형태 쪼개기
df_smallCategories = df_smallCategories2.explode("benefitdetail_name").reset_index(drop=True)
df_smallCategories['benefitdetail_id']=''
for i in range(len(df_smallCategories)):
    df_smallCategories['benefitdetail_id'].iloc[i] = i+1
df_smallCategories = df_smallCategories[["benefitdetail_id","benefitcategory_id","benefitdetail_name"]]
df_smallCategories = pd.merge(df_smallCategories, df_link, on="benefitdetail_name", how="left")

#### 마무리 가공 시작

In [ ]:
#상세혜택 데이터 삽입
#데이터 series 형태 데이터 삽입
detail_records = []
place_id_map = pd.Series(df_smallCategories.benefitdetail_id.values,
                         index=df_smallCategories.benefitdetail_name).to_dict()

#상세혜택에 cardid, 소카테고리 데이터 연결
for card in card_detail_data:
    card_id = card.get('id')
    benefits = card.get('benefits', [])

    for benefit in benefits:
        place = benefit.get("place")
        text = benefit.get("text")
        info = benefit.get("info")
        
        benefitdetail_id = place_id_map.get(place)
        
        if benefitdetail_id:
            detail_records.append({
                "card_id": card_id,
                "benefitdetail_id": benefitdetail_id,
                "cardbenefitdetail_text": text,
                "cardbenefitdetail_info": info
            })
df_card = pd.DataFrame(detail_records)        
df_card['cardbenefitdetail_id'] = range(1, len(df_card) + 1)

df_card = df_card[[
    'cardbenefitdetail_id',
    'card_id',
    'benefitdetail_id',
    'cardbenefitdetail_text',
    'cardbenefitdetail_info'
]]
import numpy as np

# NaN → None으로 변환
df_card = df_card.where(pd.notnull(df_card), '-')
print(df_card)

### DATABASE에 값 저장 시작

#### Card table 값 저장

In [ ]:
#card 값 삽입
df_card_final = df_card_final.where(pd.notnull(df_card_final), None)

# DB 연결
conn = pymysql.connect(
    host='localhost',
    user='cardgarden',
    password='1234',
    database='test',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 컬럼 9개에 맞는 INSERT 문
insert_sql = """
INSERT INTO Card (
    card_id, card_name, company, card_type, brand, card_image,
    card_url, fee_domestic, fee_foreign, prev_month_cost
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
try:
    for _, record in df_card_final.iterrows():
        cursor.execute(insert_sql, (
            record["card_id"],
            record["card_name"],
            record["company"],
            record["card_type"],
            record["brand"],
            record["card_image"],
            record["card_url"],
            record["fee_domestic"],
            record["fee_foreign"],
            record["prev_month_cost"]
        ))
    conn.commit()
except Exception as e:
    print("에러 발생:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

#### BenefitDetail table 값 저장

In [ ]:
#소카테고리 데이터 삽입
# NaN을 None으로 변환
df_card_final = df_card_final.where(pd.notnull(df_card_final), None)

conn = pymysql.connect(
    host='192.168.0.11',
    user='cardgarden',
    password='1234',
    database='cardgarden',
    charset='utf8mb4'
)
cursor = conn.cursor()

insert_sql = """
INSERT INTO BenefitDetail (benefitcategory_id, benefitdetail_name,benefitdetail_image)
VALUES (%s, %s,%s)
"""

try:
    for _, record in df_smallCategories.iterrows():
        cursor.execute(insert_sql, (
            record["benefitcategory_id"],
            record["benefitdetail_name"],
            record["benefitdetail_image"]
        ))
    conn.commit()
except Exception as e:
    print("에러 발생:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

#### CardBenefitDetail table 값 저장

In [ ]:
# NaN 또는 리스트 등을 처리하는 안전한 JSON 변환 함수
def safe_json(val):
    try:
        if val is None or (isinstance(val, float) and pd.isna(val)):
            return ''
        if isinstance(val, (list, dict)):
            return json.dumps(val, ensure_ascii=False)
        return str(val)
    except Exception as e:
        print("⚠️ 변환 실패:", e)
        return ''

# DB 연결
conn = pymysql.connect(
    host='localhost',  # ← 'local' → 'localhost'
    user='cardgarden',
    password='1234',
    database='cardgarden',
    charset='utf8mb4',
    autocommit=True  # 자동 커밋 옵션 추가
)

try:
    with conn.cursor() as cursor:
        insert_sql = """
        INSERT INTO CardBenefitDetail (
            card_id, benefitdetail_id, cardbenefitdetail_text, cardbenefitdetail_info
        )
        VALUES (%s, %s, %s, %s)
        """
        
        # 데이터프레임에서 한 줄씩 삽입
        for idx, record in df_card.iterrows():
            try:
                card_id = int(record["card_id"])
                benefitdetail_id = int(record["benefitdetail_id"])
                text = str(record["cardbenefitdetail_text"])
                info = safe_json(record["cardbenefitdetail_info"])
                
                cursor.execute(insert_sql, (card_id, benefitdetail_id, text, info))

            except Exception as e:
                print(f"❌ [삽입 실패 - {idx}번째 row]:", e)
                print("   ⤷ 내용:", record)
finally:
    conn.close()